# Ejercicio 2: Etapa de Preprocesado de Texto

In [ ]:
####################################################################
# Paso 1: Eliminación de Caracteres Especiales, Números y Minúsculas
####################################################################

import re
import pandas as pd
import os

# Definir la ruta del archivo
WORKING_PATH = "./"
DATASET_PATH = os.path.join(WORKING_PATH, "Musical_Instruments_5.json")

# Cargar el dataset
df = pd.read_json(DATASET_PATH, lines=True)
# Función de limpieza básica
def clean_text(text):
    text = text.lower()  # Convertir a minúsculas
    text = re.sub(r"\d+", "", text)  # Eliminar números
    text = re.sub(r"[^\w\s]", "", text)  # Eliminar caracteres especiales
    return text

# Aplicar la limpieza al texto
df["review_clean"] = df["reviewText"].dropna().apply(clean_text)

# Mostrar antes y después
# df[["reviewText", "review_clean"]].head()

################################################
# Paso 2: Eliminación de Stopwords
################################################

import nltk
from nltk.corpus import stopwords

# Descargar stopwords de NLTK 
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

# Función para eliminar stopwords
def remove_stopwords(text):
    words = text.split()  # Tokenizar en palabras
    words = [word for word in words if word not in stop_words]  # Filtrar stopwords
    return " ".join(words)

# Aplicar la eliminación de stopwords
df["review_clean"] = df["review_clean"].apply(remove_stopwords)

# Mostrar antes y después
# df[["reviewText", "review_clean"]].head()

#################################
# Paso 3: Lematización del Texto
#################################

import spacy

# Cargar el modelo de spaCy en inglés
nlp = spacy.load("en_core_web_sm")

# Función para lematizar el texto
def lemmatize_text(text):
    doc = nlp(text)  # Procesar el texto con spaCy
    words = [token.lemma_ for token in doc]  # Obtener las formas lematizadas
    return " ".join(words)

# Aplicar lematización al texto limpio
# df["review_clean"] = df["review_clean"].apply(lemmatize_text)

# Aplicar lematización en lotes para acelerar el procesamiento
df["review_clean"] = [" ".join([token.lemma_ for token in doc]) for doc in nlp.pipe(df["review_clean"], batch_size=5000)]


# Mostrar antes y después
# df[["reviewText", "review_clean"]].head()

#################################
# Paso 4: Tokenización Eficiente
#################################

# Aplicar tokenización con spaCy
df["tokens"] = [[token.text for token in doc] for doc in nlp.pipe(df["review_clean"], batch_size=5000)]

# Mostrar antes y después
# df[["review_clean", "tokens"]].head()

# Reclasificar las reviews: ahora 3 estrellas se consideran negativas
df["sentiment"] = df["overall"].apply(lambda x: "positive" if x >= 4 else "negative")

# Mostrar antes y después
# df[["review_clean", "tokens", "processed_text"]].head()

##########################################
# Paso 6: Guardar el DataFrame Preprocesado
##########################################

import pickle

# Guardar en CSV
df.to_csv("./Musical_Instruments_preprocessed.csv", index=False)

# Guardar en Pickle
df.to_pickle("./Musical_Instruments_preprocessed.pkl")

print("📌 Archivo guardado correctamente.")

[nltk_data] Downloading package stopwords to C:\Users\Alejandro
[nltk_data]     Catalan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


📌 Archivo guardado correctamente.


In [ ]:
#################################
# Paso 3: Lematización del Texto
#################################

import spacy

# Cargar el modelo de spaCy en inglés
nlp = spacy.load("en_core_web_sm")

# Función para lematizar el texto
def lemmatize_text(text):
    doc = nlp(text)  # Procesar el texto con spaCy
    words = [token.lemma_ for token in doc]  # Obtener las formas lematizadas
    return " ".join(words)

# Aplicar lematización al texto limpio
#df["review_clean"] = df["review_clean"].apply(lemmatize_text)

# Aplicar lematización en lotes para acelerar el procesamiento
df["review_clean"] = [" ".join([token.lemma_ for token in doc]) for doc in nlp.pipe(df["review_clean"], batch_size=5000)]

# Mostrar antes y después
df[["reviewText", "review_clean"]].head()


,reviewText,review_clean
0,Installing the game was a struggle (because of...,instal game struggle game window live bugssome...
1,If you like rally cars get this game you will ...,like rally car get game funit orient european ...
2,1st shipment received a book instead of the ga...,st shipment receive book instead gamend shipme...
3,"I got this version instead of the PS3 version,...",get version instead ps version turn mistake co...
4,I had Dirt 2 on Xbox 360 and it was an okay ga...,dirt xbox okay game start play game laptop buy...


In [4]:
#################################
# Paso 4: Tokenización Eficiente
#################################

# Aplicar tokenización con spaCy
df["tokens"] = [[token.text for token in doc] for doc in nlp.pipe(df["review_clean"], batch_size=5000)]

# Mostrar antes y después
df[["review_clean", "tokens"]].head()


,review_clean,tokens
0,instal game struggle game window live bugssome...,"[instal, game, struggle, game, window, live, b..."
1,like rally car get game funit orient european ...,"[like, rally, car, get, game, funit, orient, e..."
2,st shipment receive book instead gamend shipme...,"[st, shipment, receive, book, instead, gamend,..."
3,get version instead ps version turn mistake co...,"[get, version, instead, ps, version, turn, mis..."
4,dirt xbox okay game start play game laptop buy...,"[dirt, xbox, okay, game, start, play, game, la..."


In [16]:
##########################################################
# Paso 5: Guardar Texto Preprocesado en una Nueva Columna
##########################################################

# Unir tokens en una sola cadena de texto
df["processed_text"] = df["tokens"].apply(lambda tokens: " ".join(tokens))

# Mostrar antes y después
df[["review_clean", "tokens", "processed_text"]].head()


,review_clean,tokens,processed_text
0,instal game struggle game window live bugssome...,"[instal, game, struggle, game, window, live, b...",instal game struggle game window live bugssome...
1,like rally car get game funit orient european ...,"[like, rally, car, get, game, funit, orient, e...",like rally car get game funit orient european ...
2,st shipment receive book instead gamend shipme...,"[st, shipment, receive, book, instead, gamend,...",st shipment receive book instead gamend shipme...
3,get version instead ps version turn mistake co...,"[get, version, instead, ps, version, turn, mis...",get version instead ps version turn mistake co...
4,dirt xbox okay game start play game laptop buy...,"[dirt, xbox, okay, game, start, play, game, la...",dirt xbox okay game start play game laptop buy...


In [17]:
##########################################
# Paso 6: Guardar el DataFrame Preprocesado
##########################################

import pickle

# Guardar en CSV
df.to_csv("./video_games_reviews_preprocessed.csv", index=False)


# Guardar en Pickle
df.to_pickle("./video_games_reviews_preprocessed.pkl")

print("📌 Archivo guardado correctamente.")


# Para cargar el CSV
# df = pd.read_csv("./video_games_reviews_preprocessed.csv")

# Para cargar el Pickle
# df = pd.read_pickle("./video_games_reviews_preprocessed.pkl")

📌 Archivo guardado correctamente.
